We begin Phase III through the replication of the winning solution by Chris Detotte and Konstatin Yalenkov. The first step will be a more mature EDA, we will first follow alijs' notebook as Chris uses it as the basis of their analysis, and then replicate the innovations that Chris makes.

Link to alijs' EDA notebook: https://www.kaggle.com/alijs1/ieee-transaction-columns-reference

Link to Chris' EDA notebook: https://www.kaggle.com/cdeotte/eda-for-columns-v-and-id